In [1]:
from ipynb.fs.full.task_4 import BWT_Read_Matching
from ipynb.fs.full.task_4 import BWT

In [2]:
def generate_range(pos, p, s):
    # shifts the range of exon region from the orignal refrence string 
    # to new string(1 based index)

    # Parameters:
    # pos: pos[i][0] = strating coordinate, pos[i][1] = ending coordinate for the ith exon
    # p: previous strating positon(1 based index)
    # s: previous strating position(in original chrX)

    # Returns:
    # list containing shifted exon position in the new string

    rnge = []
    for i in range(len(pos)):

        l = pos[i][1] - pos[i][0] + 1
        x = pos[i][0]  - s

        p += x
        rnge.append([p, p + l - 1])
        p += l
        s = (pos[i][1] + 1)

    return rnge

In [3]:
red_exons_pos = [   [149249757, 149249868], 
                    [149256127, 149256423],
                    [149258412, 149258580],
                    [149260048, 149260213],
                    [149261768, 149262007],
                    [149264290, 149264400]
                ]

gree_exons_pos = [
                    [149288166, 149288277],
                    [149293258, 149293554],
                    [149295542, 149295710],
                    [149297178, 149297343],
                    [149298898, 149299137],
                    [149301420, 149301530]
                ]

In [4]:
file = open('./chrX.fa')
lines = file.readlines()
file.close()

ref_str = ""
for i in range(1, len(lines)):
    lines[i] = lines[i][:-1]
    ref_str += lines[i]

ref_str = ref_str[149249657:149301631] #extracting only the red & green exon region from the chrX with the added padding

In [5]:
red_exon_range = generate_range(red_exons_pos, 1, 149249657)
green_exon_range = generate_range(gree_exons_pos, 14744, 149264400)

In [6]:
bwt = BWT(ref_str)
brm = BWT_Read_Matching(bwt[0], bwt[1], 100, 100) 

In [7]:
def find_matching_percentage(read_pos, exon_pos):
    # finds what percentage of read length is matching in a exon region

    # Parameters:
    # read_pos: read_pos[0] = starting position of the read, read_pos[1] = ending postion of the read
    # exonpos: exon_pos[0] = starting position of the exon region, exon_pos[1] = endig postion of the exon region

    # Returns:
    # the percentage of the read length that is matching in the exon region

    if (read_pos[1] < exon_pos[0]) or (read_pos[0] > exon_pos[1]):
        return 0
    else:
        read_len = read_pos[1] - read_pos[0] + 1
        start = exon_pos[0] if read_pos[0] <= exon_pos[0] else read_pos[0]
        end = read_pos[1] if read_pos[1] <= exon_pos[1] else exon_pos[1]
        matched_len = (end - start + 1)

        percentage_matched = round((matched_len/read_len) * 100, 2)
        return percentage_matched

# MATCHING READS USING BWT READ MATCHING ALGORITHM

In [8]:
def print_read_region_match(file, threshold):
    # find how many reads matches, in each exon region with the allowed 
    # threshold percentage of the matcing read length, and reads are given in the file: file

    # Parameters:
    # file: name of the file in which reads are avialble 
    # threshold: percentage of read length that should be matched 
    # in order to consider a read to be matched in a region

    file = open(file)
    lines = file.readlines()
    file.close()

    n = len(red_exon_range)
    red_region = [0] * n
    green_region = [0] * n

    both_region = [0] * n

    nd_green_region = [0] * n
    nd_red_region = [0] *n

    r_by_g_per_d = [0]*n
    r_by_g_per_nd = [0] * n

    match_cnt = {0:0}

    for ln in lines:

        read = ln[:-1]

        l = len(read)

        nc = read.count("N")
        p = brm.find_pattern_approx(ref_str, read, 1) if nc == 0 else brm.find_pattern_approx(ref_str, read, nc + 1) #if read contains N, then match the read with number of N + 1 miss-match function
        if p[0] == -1:
            match_cnt[0] += 1
            continue

        if len(p) not in match_cnt:
            match_cnt[len(p)] = 1
        else:
            match_cnt[len(p)] += 1

        pos = []
        for i in p:
            pos.append([i, i + l - 1])

        pad = len(read)

        cnt, r1, r2 = 0, -1, -1
        for p in pos:

            for i in range(len(red_exon_range)):

                if find_matching_percentage(p, red_exon_range[i]) >= threshold:
                    red_region[i] += 1
                    cnt += 1
                    r1 = i

                if find_matching_percentage(p, green_exon_range[i]) >= threshold:
                    green_region[i] += 1
                    cnt += 1
                    r2 = i

        if cnt == 2:
            if (r1 == r2):
                both_region[r1] += 1
            if r1 == -1 or r2 == -1:
                print(read, r1, r2)
        elif cnt == 1:
            if r1 != -1:
                nd_red_region[r1] += 1
            else:
                nd_green_region[r2] += 1

    for i in range(n):
        if green_region[i] != 0:
            r_by_g_per_d[i] = round((red_region[i]/green_region[i]) * 100, 2)
        else:
            r_by_g_per_d[i] = "N/A"
        
        if nd_green_region[i] != 0:
            r_by_g_per_nd[i] = round((nd_red_region[i]/nd_green_region[i]) * 100, 2)
        else:
            r_by_g_per_nd[i] = "N/A"

    print(f"for file: {file.name}")
    print("Double counting:")
    print(f"matches in red exon region: {red_region}")
    print(f"matchse in green exon region: {green_region}")
    print(f"red to green percentage: {r_by_g_per_d}")
    print(f"\nreads that match in both exon region: {both_region}\n")
    print("Not double counting:")
    print(f"matches in red exon region: {nd_red_region}")
    print(f"matchse in green exon region: {nd_green_region}")
    print(f"red to green percentage: {r_by_g_per_nd}\n")

    for l in match_cnt:
        print(f"number of reads that matched {l} times: {match_cnt[l]}")
    print("\n")

In [9]:
print_read_region_match("reads", 25)

for file: reads
Double counting:
matches in red exon region: [239, 240, 86, 189, 371, 632]
matchse in green exon region: [267, 323, 156, 173, 433, 632]
red to green percentage: [89.51, 74.3, 55.13, 109.25, 85.68, 100.0]

reads that match in both exon region: [222, 205, 60, 83, 187, 632]

Not double counting:
matches in red exon region: [17, 35, 26, 106, 184, 0]
matchse in green exon region: [45, 118, 96, 90, 246, 0]
red to green percentage: [37.78, 29.66, 27.08, 117.78, 74.8, 'N/A']

number of reads that matched 0 times: 3058003
number of reads that matched 1 times: 6291
number of reads that matched 3 times: 1
number of reads that matched 2 times: 2425


